### Projet de Natural Language Processing : Analyse de sentiments des films de la plateforme IMDB

### Introduction et objectifs du projet

#### <u>Contexte :</u>

L'analyse de sentiments est un domaine essentiel en science des données, en particulier en NLP. Elle utilise le machine learning et la linguistique pour détecter, extraire et étudier les émotions et les informations subjectives dans les textes sources.

Dans le contexte des avis de films, l'analyse de sentiments permet de comprendre les perceptions et les émotions des spectateurs à partir de leurs commentaires, ce qui est précieux pour les critiques de films, les studios de cinéma et les plateformes de streaming.

#### <u>Objectifs :</u>

Ce projet vise à développer une application interactive en Python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques. Les objectifs de l'application sont les suivants :

1. Permettre aux utilisateurs de saisir un titre de film ou son ID pour récupérer des avis pertinents.

2. Analyser les sentiments exprimés dans ces avis, en calculant la polarité (positivité ou négativité des sentiments) et la subjectivité (mesure de l'opinion personnelle par rapport à un fait objectif) des commentaires.

3. Comparer les "ratings" (notes) des films avec la polarité des avis pour voir si les évaluations numériques correspondent aux sentiments exprimés dans les commentaires.

4. Examiner les différences, le cas échéant, entre les sentiments exprimés avant et après la sortie du film, fournissant ainsi une perspective unique sur l'évolution des perceptions des spectateurs.

#### <u>Méthodes de récupération des données :</u>

Dans un premier temps, pour éviter de surcharger l'API de TMDB et de devoir gérer des bases de données trop volumineuses, nous avons décidé de limiter notre étude aux films sortis entre x et y. Une fois cette sélection effectuée, nous avons procédé à la récupération des données via 3 méthodes différentes :

- L'API de TMDB : "https://developers.themoviedb.org/3/"
- Scraping du site : imdb.com (via la librairie python : beautifulsoup4)
- Utilisation de la librairie python : imdbpy

L'objectif de cette section est de comparer les différentes méthodes de récupération des données afin de déterminer laquelle est la plus efficace et la plus pertinente pour notre projet.
Dans une première approche il était question de tester les 3 méthodes, voir les avantages et inconvénients de chacune d'entre elles, de savoir si elles étaient complémentaires ou non, et de déterminer laquelle était la plus efficace et la plus pertinente pour notre projet.

Dans un second temps l'objectif est de structurer les données récupérées afin de pouvoir les exploiter dans la suite du projet. Si possible par une seule des méthodes de récupération des données mentionnées ci-dessus.

Ensuite, il s'agira de mettre en place une méthode d'analyse de sentiments sur les commentaires récupérés, il existe des librairies python qui permettent de faire cela, mais il est également possible de créer notre propre méthode d'analyse de sentiments. 

Choix numéro 1 : Réaliser nos propres méthodes d'analyse de sentiments.

Choix numéro 2 : Utiliser des librairies python existantes tel que : TextBlob, NLTK.

Enfin il s'agira de mettre en place une application interactive en python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques.


#### <u>Import des librairies</u>

- package imdb à importer via le code ci-dessous : (important qu'il soit installé sur la machine et qu'il soit à jour)

In [1]:
#pip install git+https://github.com/santhoshse7en/imdb

In [1]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import requests
from datetime import datetime
import json
import re
import pandas as pd

- Appel de la clé API de TMDB 

In [2]:
with open('clefAPI.json', 'r') as config_file:
    config = json.load(config_file)

api_key = config['tmdb']['api_key']

#### <u>Méthode 1 : scraping du site imdb.com </u>

Première étape : on va tester notre procédé sur un seul film, pour voir si ça fonctionne, et si oui, on pourra ensuite l'appliquer à tous les films de notre base de données.

Étape crucial : on doit récupérer la date de sortie du film, pour pouvoir ensuite récupérer les commentaires avant et après la sortie du film. 

The Shawshank Redemption : ID sur le site de TMDB : "tt0111161"

In [4]:
film_id = 'tt0111161'

url_date_release = f"https://api.themoviedb.org/3/movie/{film_id}?api_key={api_key}&language=en-US"

response = requests.get(url_date_release)

if response.status_code == 200:
    film_details = response.json()
    date_sortie_str = film_details.get("release_date", "")
    if date_sortie_str:
        date_sortie = datetime.strptime(date_sortie_str, "%Y-%m-%d")
        print(f"Date de sortie du film : {date_sortie}")
    else:
        print("Date de sortie non trouvée")
else:
    print("Erreur lors de la requête :", response.status_code)

url_film = f"https://www.imdb.com/title/{film_id}/reviews" # Récupérer les avis sur IMDb

response = requests.get(url_film)

reviews_dict = {}

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    review_elements = soup.find_all('div', class_='review-container')
    for element in review_elements:
        review_text = element.find('div', class_='text show-more__control').get_text().strip()
        review_date_str = element.find('span', class_='review-date').get_text().strip()
        review_date = datetime.strptime(review_date_str, '%d %B %Y').date()
        reviews_dict[review_date] = review_text
else:
    print("Erreur lors de la requête :", response.status_code)

print("Nombre d'avis pour ce film :",len(reviews_dict)) 

Date de sortie du film : 1994-09-23 00:00:00
Nombre d'avis pour ce film : 25


- Objectif : voir si on a assez de commentaires avant et après la sortie du film pour pouvoir faire une analyse de sentiments.

In [5]:
avis_avant = {}
avis_apres = {}

for review_date, review_text in reviews_dict.items():
    if review_date < date_sortie.date():
        avis_avant[review_date] = review_text
    else:
        avis_apres[review_date] = review_text

print("Nombre d'avis avant la sortie du film :", len(avis_avant))
print("Nombre d'avis après la sortie du film :", len(avis_apres))

Nombre d'avis avant la sortie du film : 0
Nombre d'avis après la sortie du film : 25


On constate alors qu'avant la sortie du film, on a 0 commentaires, et après la sortie du film, on a 25 commentaire. On ne peut donc pas faire d'analyse de sentiments sur ce film. Et en testant sur d'autres films, on se rend compte que c'est le cas pour tous des films de notre base de données. Afin de réaliser le projet dans son intégralité, il faudrait donc récupérer les commentaires sur une autre plateforme que TMDB. Toutefois nous manquons de temps et d'expérience pour réaliser cela. Nous allons donc nous contenter de réaliser une analyse de sentiments sur les commentaires récupérés sur TMDB.

- Affichage des 5 premiers commentaires récupérés sur TMDB pour ce premier film :

In [6]:
print("Avis après la sortie:")
for date, review in list(avis_apres.items())[:5]:
    print(f"{date}: {review}\n")

Avis après la sortie:
2010-07-24: The Shawshank Redemption is written and directed by Frank Darabont. It is an adaptation of the Stephen King novella Rita Hayworth and Shawshank Redemption. Starring Tim Robbins and Morgan Freeman, the film portrays the story of Andy Dufresne (Robbins), a banker who is sentenced to two life sentences at Shawshank State Prison for apparently murdering his wife and her lover. Andy finds it tough going but finds solace in the friendship he forms with fellow inmate Ellis "Red" Redding (Freeman). While things start to pick up when the warden finds Andy a prison job more befitting his talents as a banker. However, the arrival of another inmate is going to vastly change things for all of them.There was no fanfare or bunting put out for the release of the film back in 94, with a title that didn't give much inkling to anyone about what it was about, and with Columbia Pictures unsure how to market it, Shawshank Redemption barely registered at the box office. Howe

#### <u>Méthode 2 : utilisation de l'API </u>

In [41]:
def get_movies_between_dates(start_date, end_date, api_key):
    base_url = "https://api.themoviedb.org/3/discover/movie"
    url = f"{base_url}?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&region=FR"
    response = requests.get(url)
    if response.status_code == 200:
        movies_data = response.json()['results']
        return [(movie['id'], movie['title']) for movie in movies_data]
    else:
        print("Erreur lors de la requête :", response.status_code)
        return []
    
start_date = "2022-01-01" # FONCTIONNE ! On peut donc définir la période de recherche que l'on souhaite
end_date = "2022-01-31" 

print(f"Recherche des films sortis entre le {start_date} et le {end_date}", end=" ")
print(get_movies_between_dates(start_date, end_date, api_key))

Recherche des films sortis entre le 2022-01-01 et le 2022-12-31 [(76600, 'Avatar: The Way of Water'), (457232, 'Lamborghini: The Man Behind the Legend'), (315162, 'Puss in Boots: The Last Wish'), (606870, 'The Survivor'), (436270, 'Black Adam'), (833326, 'Zero Fucks Given'), (414906, 'The Batman'), (505642, 'Black Panther: Wakanda Forever'), (675353, 'Sonic the Hedgehog 2'), (361743, 'Top Gun: Maverick'), (1028318, 'Goodbye Monster'), (585083, 'Hotel Transylvania: Transformania'), (453395, 'Doctor Strange in the Multiverse of Madness'), (507086, 'Jurassic World Dominion'), (438148, 'Minions: The Rise of Gru'), (829560, 'The Next 365 Days'), (616037, 'Thor: Love and Thunder'), (760104, 'X'), (899112, 'Violent Night'), (744276, 'After Ever Happy')]


<font color="red">NE PAS RELANCER SINON ECRASEMENT DE MON DICTIONNAIRE ET 7 MIN DE LANCEMENT</font>

In [7]:
def get_movies_between_dates(start_date, end_date, api_key):
    base_url = "https://api.themoviedb.org/3/discover/movie"
    movies = []
    page = 1
    total_pages = 1

    while page <= total_pages:
        url = f"{base_url}?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&region=FR&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            total_pages = data['total_pages']
            movies.extend([(movie['id'], movie['title']) for movie in data['results']])
            page += 1
        else:
            print("Erreur lors de la requête :", response.status_code)
            break
    return movies

def get_movie_reviews(movie_id, api_key):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['results']
    else:
        print(f"Erreur lors de la récupération des avis pour {movie_id}: {response.status_code}")
        return None

def filter_movies_with_reviews(movies, api_key):
    movies_with_reviews = []
    for movie_id, movie_title in movies:
        reviews = get_movie_reviews(movie_id, api_key)
        if reviews and len(reviews) > 0:
            movies_with_reviews.append((movie_id, movie_title))
    return movies_with_reviews


start_date = "2022-01-01"
end_date = "2022-03-31"

print(f"Recherche des films sortis entre le {start_date} et le {end_date}")

movies = get_movies_between_dates(start_date, end_date, api_key)
movies_with_reviews = filter_movies_with_reviews(movies, api_key)

print(f"Nombre de films avec des avis de {start_date} à {end_date} : {len(movies_with_reviews)}")

print(f"Nombre de films trouvés : {len(movies)}")

Recherche des films sortis entre le 2022-01-01 et le 2022-03-31
Nombre de films avec des avis de 2022-01-01 à 2022-03-31 : 83
Nombre de films trouvés : 925


#### <u>Méthode 3 : utilisation de la librairie python : imdbpy</u>

In [ ]:
# from cinemagoer import IMDb
# ia = IMDb()
# movie = ia.get_movie('0111161')  # Pour 'The Shawshank Redemption'
# print(movie['title'])  # Afficher le titre du film
#!pip show IMDbPY

#### <u>Récupération des données via la méthode de scraping du site imdb.com</u>

In [3]:
def get_imdb_reviews(movie_id): # FONCTIONNE
    url = f"https://www.imdb.com/title/{movie_id}/reviews"
    response = requests.get(url)
    reviews = {}
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', class_='review-container')
        if not review_elements:
            print(f"Aucune critique trouvée pour {movie_id}")
            return reviews
        for element in review_elements:
            review_text = element.find('div', class_='text show-more__control').get_text().strip()
            review_date = datetime.strptime(element.find('span', class_='review-date').get_text(), '%d %B %Y')
            reviews[review_date] = review_text
    else:
        print(f"Erreur lors de la récupération des avis pour {movie_id}: {response.status_code}")
    return reviews

print(get_imdb_reviews('tt0111161'))

Erreur lors de la récupération des avis pour tt0111161: 500
{}


#### <u>Récupération des données via l'API de TMDB</u>

In [11]:
def get_movie_details(movie_id, api_key):
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Erreur lors de la requête :", response.status_code)
        return None

print(get_movie_details('tt0111161', api_key))

{'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'belongs_to_collection': None, 'budget': 25000000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}], 'homepage': '', 'id': 278, 'imdb_id': 'tt0111161', 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 110.352, 'poster_path': '/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg', 'production_companies': [{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxYqAlPPykp.png', 'name': 'Castle Rock Entertainment', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'Unit

In [4]:
def get_movie_details_and_reviews(movie_id, api_key):
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    movie_details = response.json() if response.status_code == 200 else {}
    reviews_url = f"{base_url}{movie_id}/reviews?api_key={api_key}"
    reviews_response = requests.get(reviews_url)
    reviews_data = reviews_response.json() if reviews_response.status_code == 200 else {}
    return movie_details, reviews_data

#print(get_movie_details_and_reviews('tt0111161', api_key))

({'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'belongs_to_collection': None, 'budget': 25000000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}], 'homepage': '', 'id': 278, 'imdb_id': 'tt0111161', 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 141.475, 'poster_path': '/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg', 'production_companies': [{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxYqAlPPykp.png', 'name': 'Castle Rock Entertainment', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'Uni

#### <u>On transforme nos données en json</u>

In [8]:
movies_details = {}

for movie_id, movie_title in movies_with_reviews:
    movie_details, reviews_data = get_movie_details_and_reviews(movie_id, api_key)
    movies_details[movie_id] = {
        'title': movie_title,
        'details': movie_details,
        'reviews': reviews_data
    }

# Ne lancer qu'une seule fois pour éviter d'écraser le fichier JSON
# with open('movies_details_3mois.json', 'w') as json_file: 
#    json.dump(movies_details, json_file, indent=4)

Maintenant que nous avons récupérées toutes les données utiles à notre projet dans un fichier "movies_details.json", nous allons pouvoir les exploiter.

In [9]:
movies_details

{414906: {'title': 'The Batman',
  'details': {'adult': False,
   'backdrop_path': '/tRS6jvPM9qPrrnx2KRp3ew96Yot.jpg',
   'belongs_to_collection': {'id': 948485,
    'name': 'The Batman Collection',
    'poster_path': '/tuzKA9K5Ae9IzaA0R9oAgbyhAI3.jpg',
    'backdrop_path': '/qS2ViQwlWUECiAdkIuEaJZoq0QW.jpg'},
   'budget': 185000000,
   'genres': [{'id': 80, 'name': 'Crime'},
    {'id': 9648, 'name': 'Mystery'},
    {'id': 53, 'name': 'Thriller'}],
   'homepage': 'https://www.thebatman.com',
   'id': 414906,
   'imdb_id': 'tt1877830',
   'original_language': 'en',
   'original_title': 'The Batman',
   'overview': 'In his second year of fighting crime, Batman uncovers corruption in Gotham City that connects to his own family while facing a serial killer known as the Riddler.',
   'popularity': 167.287,
   'poster_path': '/74xTEgt7R36Fpooo50r9T25onhq.jpg',
   'production_companies': [{'id': 101405,
     'logo_path': None,
     'name': '6th & Idaho',
     'origin_country': 'US'},
    {'id

Avant de se lancer dans l'analyse de sentiments, nous allons d'abord faire des vérifications sur les données inscritent dans notre fichier json. Nous allons requêter ces données pour voir si elles sont bien présentes et si elles sont bien au format attendu.

On peut par exemple demander le nombre de films dans notre json :

In [10]:
with open('movies_details_3mois.json', 'r') as json_file:
    movies_details = json.load(json_file)

num_films = len(movies_details)

print("Nombres de  films:", num_films)

Nombres de  films: 83


On peut également demander le nombre d'avis pour chaque film :

In [11]:
try:
    with open('movies_details_3mois.json', 'r') as file:
        movies_details = json.load(file)
    reviews_count = {}
    for movie in movies_details.values():
        num_reviews = len(movie['reviews']['results'])
        movie_id = movie['reviews']['id']
        reviews_count[movie_id] = num_reviews
    for movie_id, count in reviews_count.items():
        print(f"Film ID {movie_id}: Nombre d'avis = {count}")
except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

Film ID 414906: Nombre d'avis = 20
Film ID 675353: Nombre d'avis = 7
Film ID 585083: Nombre d'avis = 1
Film ID 760104: Nombre d'avis = 5
Film ID 508947: Nombre d'avis = 11
Film ID 406759: Nombre d'avis = 6
Film ID 818647: Nombre d'avis = 1
Film ID 335787: Nombre d'avis = 8
Film ID 646385: Nombre d'avis = 9
Film ID 629542: Nombre d'avis = 5
Film ID 545611: Nombre d'avis = 17
Film ID 526896: Nombre d'avis = 9
Film ID 752623: Nombre d'avis = 8
Film ID 774825: Nombre d'avis = 1
Film ID 505026: Nombre d'avis = 9
Film ID 557946: Nombre d'avis = 2
Film ID 628900: Nombre d'avis = 3
Film ID 924482: Nombre d'avis = 1
Film ID 698508: Nombre d'avis = 1
Film ID 823625: Nombre d'avis = 1
Film ID 619979: Nombre d'avis = 3
Film ID 579974: Nombre d'avis = 5
Film ID 696806: Nombre d'avis = 8
Film ID 799876: Nombre d'avis = 5
Film ID 522016: Nombre d'avis = 6
Film ID 854867: Nombre d'avis = 1
Film ID 763285: Nombre d'avis = 8
Film ID 787752: Nombre d'avis = 3
Film ID 833339: Nombre d'avis = 2
Film ID 626

Maintenant on test d'afficher les avis sur un film en particulier : je prends par exemple l'ID du film x : "646385" et je récupère les avis sur ce film :

In [ ]:
fichier_json = 'movies_details_3mois.json'
movie_id_to_check = 646385

try:
    with open(fichier_json, 'r') as file:
        movies_details = json.load(file)

    selected_movie = movies_details[str(movie_id_to_check)]
    reviews = selected_movie['reviews']['results']
    
    for review in reviews:
        print(f"Auteur : {review['author']}")
        print(f"Rating : {review['author_details']['rating']}")
        print(f"Contenu : {review['content']}\n")

except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

## Idées pour la suite du projet :

- Créer l'application interactive en python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques.
- Analyser les mots clés les plus utilisés dans les commentaires récupérés sur TMDB.
- Réaliser un word cloud avec les mots clés les plus utilisés dans les commentaires récupérés sur TMDB.
- Faire une analyse de sentiments sur les commentaires récupérés sur TMDB.
- Faire une moyenne de notes de sentiments pour chaque film.
- Comparer les notes de sentiments des avis avec les notes de films attribuées par les utilisateurs.


#### <u> Analyse des mots clés les plus utilisés dans les commentaires récupérés sur TMDB</u>

- On va d'abord créer une fonction clean_avis qui va nous permettre de nettoyer les commentaires récupérés sur TMDB. On va supprimer les caractères spéciaux, les mots vides et on va mettre tous les mots en minuscule. Pour cela on va s'aider de la librairie nltk et de son module stopwords et de la librairie re.

- Ensuite on va construire une fonction qui va nous permettre de compter le nombre d'occurence de chaque mot dans les commentaires. On utilisera notamment la fonction Counter de la librairie collections.

In [15]:
import json

from collections import Counter
import re
from nltk.corpus import stopwords
import nltk

#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_avis(text):
    mots_a_retirer = ['film', 'movie']
    all_stopwords = stop_words.union(set(mots_a_retirer))
    all_stopwords = list(stop_words) + mots_a_retirer
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    words = [word for word in text.split() if word not in all_stopwords]
    return words

def analyse_word_freq(movie_reviews):
    all_words = []
    for review in movie_reviews:
        all_words.extend(clean_avis(review['content']))
    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(10)
    return most_common_words

with open('movies_details_3mois.json', 'r') as file:
    movies_details = json.load(file)
    

word_freq_film = {}

for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    most_common_words = analyse_word_freq(reviews)
    word_freq_film[movie_id] = most_common_words

for movie_id, frequencies in word_freq_film.items():
    #print(f"Film ID {movie_id}: Mots les plus fréquents - {frequencies}")

    print(f"{movies_details[movie_id]['title']}\nMots les plus fréquents {frequencies}\n")

The Batman
Mots les plus fréquents [('batman', 123), ('like', 32), ('riddler', 23), ('also', 22), ('even', 18), ('pattinson', 17), ('bruce', 17), ('dark', 17), ('much', 17), ('best', 16)]

Sonic the Hedgehog 2
Mots les plus fréquents [('sonic', 20), ('first', 8), ('carrey', 8), ('one', 7), ('jim', 7), ('hedgehog', 7), ('robotnik', 6), ('well', 6), ('review', 6), ('fun', 6)]

Hotel Transylvania: Transformania
Mots les plus fréquents [('hotel', 2), ('transylvania', 2), ('transformania', 2), ('review', 2), ('lot', 1), ('obstacles', 1), ('relating', 1), ('release', 1), ('viewing', 1), ('understand', 1)]

X
Mots les plus fréquents [('x', 6), ('horror', 6), ('west', 5), ('story', 4), ('goth', 4), ('pearl', 4), ('makes', 4), ('gore', 3), ('mia', 3), ('interesting', 3)]

Turning Red
Mots les plus fréquents [('red', 11), ('pixar', 10), ('one', 9), ('turning', 8), ('mei', 8), ('animation', 6), ('panda', 6), ('puberty', 6), ('going', 6), ('characters', 6)]

Moonfall
Mots les plus fréquents [('one

Après réfléxion, au vu du peu d'avis récupérés sur TMDB voir même de l'absence d'avis pour certains films, nous avons décidé d'élargir notre base de données en récupérant les données des films sur une plus large période. Nous avons donc décidé de récupérer les données des films sortis pendant l'année 2022 sur les mois de X à Y en retirant les films qui n'avaient pas d'avis sur TMDB.

<font color="red">MODIFIER DATES</font>

<font color="red">PENSER A RETIRER LES MOTS INUTILES, IL Y EN A ENCORE APRES LE CLEAN/font>


Création et génération des WordClouds :

In [16]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def create_wordcloud(text, movie_id):
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stop_words, 
                min_font_size = 10).generate(' '.join(text))
    wordcloud.to_file(f"static/wordclouds/{movie_id}.png")
    
for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    review_text = [review['content'] for review in reviews]
    cleaned_text = []
    for text in review_text:
        cleaned_text.extend(clean_avis(text))
    create_wordcloud(cleaned_text, movie_id)

#### <u> Analyse de sentiments sur les commentaires récupérés sur TMDB</u>

In [17]:
from textblob import TextBlob

def analyse_sentiments(reviews):
    sentiments = []
    for review in reviews:
        blob = TextBlob(review['content'])
        sentiments.append((blob.sentiment.polarity, blob.sentiment.subjectivity))
    return sentiments

with open('movies_details_3mois.json', 'r') as file:
    movies_details = json.load(file)

sentiments_film = {}

for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    sentiments = analyse_sentiments(reviews)
    sentiments_film[movie_id] = {
        'polarity': sum([s[0] for s in sentiments]) / len(sentiments) if sentiments else 0,
        'subjectivity': sum([s[1] for s in sentiments]) / len(sentiments) if sentiments else 0
    }

In [ ]:
sentiments_film

# plutot faire des notes par avis

#### <u> Création de l'application interactive + maj en continue</u>

In [19]:
from flask import Flask, render_template, request
import json
from textblob import TextBlob

def analyse_sentiment_avis(text): # ICI ON VA POUVOIR ANALYSER LE SENTIMENT D'UN AVIS 
    analysis = TextBlob(text)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity


app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    with open('movies_details_3mois.json', 'r') as file:
        movies_details = json.load(file)

    movies_list = [(movie['reviews']['id'], movie['title']) 
                for movie in movies_details.values() 
                if 'reviews' in movie and 'results' in movie['reviews'] and movie['reviews']['results']]


    selected_movie_id = None
    selected_movie_title = None
    selected_movie_details = None
    top_avis = []

    if request.method == 'POST':
        selected_movie_id = request.form.get('movie')
        selected_movie_title = next((title for id, title in movies_list if id == selected_movie_id), None)
        if selected_movie_id:
            selected_movie_details = movies_details.get(selected_movie_id, {})
            avis_list = selected_movie_details.get('reviews', {}).get('results', [])[:5]
            for avis in avis_list:
                sentiment_polarity, sentiment_subjectivity = analyse_sentiment_avis(avis['content'])
                top_avis.append({
                    'content': avis['content'],
                    'rating': avis.get('author_details', {}).get('rating'),
                    'sentiment': {
                        'polarity': sentiment_polarity,
                        'subjectivity': sentiment_subjectivity
                    }
                })

    return render_template('index.html', movies_list=movies_list, 
                        selected_movie_id=selected_movie_id, 
                        selected_movie_title=selected_movie_title,
                        movie_details=selected_movie_details,
                        top_avis=top_avis)


@app.route('/film/<int:movie_id>')
def film(movie_id):
    with open('movies_details_3mois.json', 'r') as file:
        movies_details = json.load(file)
        
    movie_details = movies_details.get(str(movie_id), {})
    movie_sentiments = sentiments_film.get(str(movie_id), {})

    return render_template('film.html', 
                        movie_details=movie_details, 
                        movie_sentiments=movie_sentiments)


if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Dec/2023 19:24:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2023 19:24:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Dec/2023 19:24:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2023 19:24:47] "GET /static/wordclouds/818647.png HTTP/1.1" 200 -


## NEW PROJET 

Voir les genres :

In [ ]:
def extract_unique_genres(file_path):
    with open(file_path, 'r') as file:
        movies_details = json.load(file)

    unique_genres = set()
    for movie in movies_details.values():
        genres = movie.get('details', {}).get('genres', [])
        for genre in genres:
            unique_genres.add(genre['name'])

    return unique_genres

genres_uniques = extract_unique_genres('movies_details_3mois.json')
print("Genres uniques :", genres_uniques)

- Calcul de la Polarité Moyenne pour Chaque Film : Pour chaque film, calculez la polarité moyenne de tous ses avis. Vous pourriez avoir besoin d'une fonction d'analyse de sentiments si vous ne l'avez pas déjà.

- Association des Films aux Genres : Créez un dictionnaire où chaque clé est un genre et chaque valeur est une liste de tuples contenant l'ID du film, le titre du film, et sa polarité moyenne.

- Sélection du Top 3 des Films par Genre : Pour chaque genre, triez les films en fonction de leur polarité moyenne et sélectionnez les trois premiers.

In [22]:
from textblob import TextBlob 

def analyse_sentiment(text):
    return TextBlob(text).sentiment.polarity

def calculate_movie_polarity(movie_reviews):
    total_polarity = 0
    num_reviews = len(movie_reviews)

    for review in movie_reviews:
        total_polarity += analyse_sentiment(review['content'])

    return total_polarity / num_reviews if num_reviews > 0 else 0

def genre_top_movies(file_path):
    with open(file_path, 'r') as file:
        movies_details = json.load(file)

    genre_movies = {genre: [] for genre in genres_uniques}

    for movie_id, movie in movies_details.items():
        movie_polarity = calculate_movie_polarity(movie['reviews']['results'])
        movie_genres = [genre['name'] for genre in movie['details']['genres']]

        for genre in movie_genres:
            if genre in genre_movies:
                genre_movies[genre].append((movie_id, movie['title'], movie_polarity))

    top_movies_per_genre = {}

    for genre, movies in genre_movies.items():
        top_movies = sorted(movies, key=lambda x: x[2], reverse=True)[:3]
        top_movies_per_genre[genre] = top_movies

    return top_movies_per_genre


top_movies = genre_top_movies('movies_details_3mois.json')
print(top_movies)

{'Fantasy': [('919355', 'Dragon Knight', 0.3192727411477411), ('585083', 'Hotel Transylvania: Transformania', 0.21666666666666665), ('854867', 'Werewolf Castle', 0.18582535885167467)], 'Mystery': [('680829', 'Master', 0.3145652173913044), ('646385', 'Scream', 0.2464593214865157), ('799876', 'The Outfit', 0.19018307296568165)], 'Animation': [('931034', 'Cat Burglar', 0.22678571428571428), ('585083', 'Hotel Transylvania: Transformania', 0.21666666666666665), ('629542', 'The Bad Guys', 0.21539164177736833)], 'TV Movie': [('892869', 'Cider and Sunsets', -0.07986111111111112)], 'Music': [('396194', 'Ennio', 0.2652173913043478), ('615904', 'Marry Me', 0.17956410415318158), ('826953', 'Better Nate Than Ever', 0.10303030303030303)], 'Science Fiction': [('696806', 'The Adam Project', 0.20097760154892272), ('664996', 'Apollo 10½:  A Space Age Childhood', 0.19246272246272245), ('545611', 'Everything Everywhere All at Once', 0.17007664395926037)], 'Action': [('919355', 'Dragon Knight', 0.319272741

In [38]:
from flask import Flask, render_template, request
import json


app = Flask(__name__)

def polarity_to_stars(polarity):
    return round(polarity * 8)

app.jinja_env.globals.update(polarity_to_stars=polarity_to_stars)

top_movies 

@app.route('/', methods=['GET', 'POST'])
def index():
    genres = list(top_movies.keys())
    selected_genre = request.form.get('genre')
    top_movies_genre = top_movies.get(selected_genre, []) if selected_genre else []

    return render_template('index2.html', genres=genres, top_movies_genre=top_movies_genre, selected_genre=selected_genre)

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Dec/2023 20:34:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2023 20:34:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2023 20:35:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2023 20:35:52] "POST / HTTP/1.1" 200 -
